In [1]:
#importing required libraries
import pandas as pd
from urllib.parse import urlparse
import re

In [2]:
#Loading the dataset
df_main = pd.read_csv("C:/Users/Insight/Desktop/Python_analysis/startup_funding.csv")
df = df_main.copy()
df.head()

,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN


In [3]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3044 entries, 0 to 3043
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Sr No              3044 non-null   int64 
 1   Date dd/mm/yyyy    3044 non-null   object
 2   Startup Name       3044 non-null   object
 3   Industry Vertical  2873 non-null   object
 4   SubVertical        2108 non-null   object
 5   City  Location     2864 non-null   object
 6   Investors Name     3020 non-null   object
 7   InvestmentnType    3040 non-null   object
 8   Amount in USD      2084 non-null   object
 9   Remarks            419 non-null    object
dtypes: int64(1), object(9)
memory usage: 237.9+ KB


### Cleaning all columns

In [4]:
#Renaming column names for consistency
def clean_column_names(df):
    df.columns = df.columns.str.replace('  ', ' ', regex=False)
    df.rename(columns = {'Sr No':'serial_no', 'Date dd/mm/yyyy':'disbursement_date', 'InvestmentnType':'investment_type'}, 
              inplace=True)
    df.columns = df.columns.str.replace(' ', '_').str.lower()
    return df

# Apply the function to clean column names
df = clean_column_names(df)
df.head()

,serial_no,disbursement_date,startup_name,industry_vertical,subvertical,city_location,investors_name,investment_type,amount_in_usd,remarks
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN


In [5]:
#Checking columns with null values
df.isnull().sum()   

serial_no               0
disbursement_date       0
startup_name            0
industry_vertical     171
subvertical           936
city_location         180
investors_name         24
investment_type         4
amount_in_usd         960
remarks              2625
dtype: int64

In [6]:
df.isnull().mean()*100  

serial_no             0.000000
disbursement_date     0.000000
startup_name          0.000000
industry_vertical     5.617608
subvertical          30.749014
city_location         5.913272
investors_name        0.788436
investment_type       0.131406
amount_in_usd        31.537451
remarks              86.235217
dtype: float64

In [7]:
df = df.drop(columns=['remarks', 'subvertical']) 

In [8]:
#Selecting columns with null values
columns_to_fill = ['industry_vertical', 'city_location', 'investors_name', 'investment_type']

#Replacing the blank and NaN columns with "Unknown" 
df[columns_to_fill] = df[columns_to_fill].fillna('Unknown')
df.head()

,serial_no,disbursement_date,startup_name,industry_vertical,city_location,investors_name,investment_type,amount_in_usd
0,1,09/01/2020,BYJU’S,E-Tech,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000"
1,2,13/01/2020,Shuttl,Transportation,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394"
2,3,09/01/2020,Mamaearth,E-commerce,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860"
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000"
4,5,02/01/2020,Fashor,Fashion and Apparel,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000"


In [9]:
#List of unwanted characters/patterns to remove
unwanted_patterns = [r'\\xc2\\xa0', r'\\xc3\\x98', r'\\xe2\\x80\\x99', r'\\xc3\\xa9c', r'\\xe2\\x80\\x93', r'\\n', r'\\x',]

#Combining patterns into one big regex pattern (handles all unwanted characters at a go)
combined_pattern = re.compile('|'.join(unwanted_patterns))

#Function to clean unwanted patterns and also handle any encoding issues
def clean_text(text):
    if isinstance(text, str):
        text = bytes(text, 'utf-8').decode('unicode_escape') #This decodes escape sequences (such as \\xc2\\xa0), unwanted patterns into their respective characters (e.g., the non-breaking space character). 
        text = combined_pattern.sub('', text) #Removing unwanted patterns
        text = re.sub(r'\s+', ' ', text).strip() #Removing extra spaces that might have been created during cleaning
    return text

#Applying the cleaning function to all columns
df = df.applymap(clean_text)

specific_rows = [2605, 2609, 2611, 2227, 1186, 141]
df_selected = df.loc[specific_rows]
df_selected.head()

,serial_no,disbursement_date,startup_name,industry_vertical,city_location,investors_name,investment_type,amount_in_usd
2605,2606,09/07/2015,Notesgen,Online Study Notes Marketplace,New Delhi,"Rajeev Saraf, Arvind Jha, R. Satya Narayanan",Seed Funding,N/A
2609,2610,13/07/2015,Ameyo,Contact Center Software Platform,Gurgaon,Forum Synergies PE Fund,Private Equity,"5,000,000"
2611,2612,13/07/2015,Satvacart,Online Grocery Delivery,Gurgaon,Palaash Ventures,Seed Funding,N/A
2227,2228,11/11/2015,Gingercrush,Product Customization Platform,Vadodara,TV Mohandas Pais family office,Seed Funding,NaN
1186,1187,30/12/2016,InstantPay,Consumer Internet,New Delhi,"RB Investments, Kaleden Holdings",Private Equity,NaN


### Cleaning the date column

In [10]:
#Function to clean and standardize dates

def clean_date(date):
    date = date.replace('.', '/').replace('//', '/') #Replacing incorrect separators
    
    #Fixing short years (e.g., '015' -> '2015')
    if date.count('/') == 2:
        day, month, year = date.split('/')
        if len(year) < 4:
            year = '20' + year[-2:]
        date = f"{day}/{month}/{year}"
    
    #Handling joined month and year (e.g., '05/072018' -> '05/07/2018')
    elif date.count('/') == 1 and len(date.split('/')[1]) == 6:
        day, month_year = date.split('/')
        month, year = month_year[:2], month_year[2:]
        date = f"{day}/{month}/{year}"

    #Converting to datetime and format as dd/mm/yyyy
    return pd.to_datetime(date, errors='coerce', dayfirst=True).strftime('%d/%m/%Y')

#Applying the function to clean and format the dates
df['disbursement_date'] = df['disbursement_date'].apply(clean_date)

specific_rows = [2571, 192, 2775, 2776, 3029, 2831, 3011]
df_selected = df.loc[specific_rows]
df_selected.head()

,serial_no,disbursement_date,startup_name,industry_vertical,city_location,investors_name,investment_type,amount_in_usd
2571,2572,01/07/2015,HandyHome,Hyperlocal Electronics repair Service,Mumbai,"Bessemer Ventures, Kae Capital",Seed Funding,"6,30,000"
192,193,05/07/2018,Loan Tap,Consumer Internet,Mumbai,"Shunwei Capital, Tuscan Ventures, Ashish Goenk...",Private Equity,"62,50,000"
2775,2776,12/05/2015,Mobiefit,Mobile Fitness App,Goa,Prototyze,Seed Funding,NaN
2776,2777,12/05/2015,Plancess,Online Education Platform,Mumbai,"Rajendra Gogri, Chandrakant Gogri",Seed Funding,"20,00,000"
3029,3030,22/01/2015,Corporate360,Unknown,Unknown,Group of Angel Investors,Seed Funding,"2,00,000"


### Cleaning the startup_name column

In [11]:
#Inspecting column
#pd.set_option("display.max_rows", None)
value_counts = df['startup_name'].value_counts().sort_index()
print(value_counts)

"BYJU\'S"            1
#Fame                1
121Policy            1
19th mile            1
1Crowd               1
                    ..
vImmune              1
vPhrase Analytics    1
wWhere               1
wayForward           1
zippserv             1
Name: startup_name, Length: 2456, dtype: int64


In [12]:
##Extracting the domain name from URL
#Function to extract domain name if the entry is a URL and capitalize the first letter
# Define the function for cleaning startup names
def clean_startup_name(value):
    # Remove non-ASCII characters
    value = value.encode('ascii', 'ignore').decode('utf-8')
    # Standardize specific patterns like "Byju's"
    value = re.sub(r"\"?Byju\\?'?S\"?", "Byjus", value, flags=re.IGNORECASE)
    
    # Check if the value is a URL
    if value.startswith('http'):
        domain_name = urlparse(value).netloc.split('.')[1].capitalize()  # Extract and capitalize domain
        return domain_name
    
    # Remove specific patterns for non-URL values
    cleaned_value = re.sub(r'\.com|\.co|\.in|\.ai|\.IO', '', value, flags=re.IGNORECASE)
    return cleaned_value.title()

# Apply the combined cleaning function to the 'startup_name' column
df['startup_name'] = df['startup_name'].apply(clean_startup_name)
df.head()

# Select specific rows
specific_rows = [2269, 2873, 1677, 1411]
df_selected = df.loc[specific_rows]
df_selected.head()

,serial_no,disbursement_date,startup_name,industry_vertical,city_location,investors_name,investment_type,amount_in_usd
2269,2270,25/11/2015,Earlysalary,Mobile App based Loan disbursement platform,Mumbai,Ashok Agarwal,Seed Funding,"15,00,000"
2873,2874,02/03/2015,Trulymadly,Unknown,Unknown,"Helion Venture Partners, Kae Capital",Private Equity,"55,00,000"
1677,1678,04/05/2016,Active,Technology,Singapore,Kalaari Capital Accelerator Program,Seed Funding,"5,00,000"
1411,1412,30/09/2016,Kalaage,Consumer Internet,Jaipur,Rajasthan Angel Innovators Network,Seed Funding,NaN


In [13]:
startup_mapping = {
    '1Mg (Healthkartplus)': '1Mg', 'Aahaa': 'Aahaa Stores', 'Absentia': 'Absentia Vr', 'Active': 'Active Ai',
    '#Fame': 'Fame', 'Arcatron': 'Arcatron Mobility', 'Ampere': 'Ampere Vehicles', 'Atomberg': 'Atomberg Technology',
    'Availfinance': 'Avail Finance', 'Ayefinance': 'Aye Finance', 'Appdaily': 'Appsdaily', 'Bank Bazaar': 'Bankbazaar',
    'Bhive': 'Bhive Workspace', 'Bhiveworkspace': 'Bhive Workspace', 'BigBasket': 'Bigbasket',
    'Black White Orange': 'Black White Orange Brands', 'Buildkar': 'Buildzar', 'BYJUs': 'Byjus', 'BYJUS': 'Byjus',
    'CCavenue': 'CCAvenue', 'Capillary': 'Capillary Tech', 'Caravan Craft Retail': 'Caravan Craft',
    'Chaipoint': 'Chai Point', 'Cloudcherry': 'Cloudcherry Analytics', 'Confirmttkt': 'Confirmtkt',
    'Craftstvilla': 'Craftsvilla', 'Crown-It': 'Crownit', 'Cult': 'Cult Fitness', 'Cure Fit': 'Curefit',
    'Daily Hunt': 'Dailyhunt', 'Daily Rounds': 'Dailyrounds', 'Deal 4Loans': 'Deal4Loans', 'Dhruva': 'Dhruva Space',
    'Doctorinsta': 'Doctor Insta', 'Early Salary': 'Earlysalary', 'Entropika': 'Entropik', 'Fab Hotels': 'Fabhotels',
    'Fyle Technologies': 'Fyle', 'Gibbs': 'Gibss', 'Glamstudios': 'Glam Studios', 'Hansel Io': 'Hansel',
    'Happilyunmarried': 'Happily Unmarried', 'Healthcare': 'Healthcare At Home', 'High Radius': 'Highradius',
    'Hike Messenger': 'Hike', 'Hwell24': 'Hwell24 Plus', 'I2I Funding': 'I2Ifunding', 'Impact Guru': 'Impactguru',
    'Incred': 'Incred Finance', 'Ink Monk': 'Inkmonk', 'Intelligencenode': 'Intelligence Node',
    'Jollyfoodfellow': 'Jolly Food Fellow', 'Legal Raasta': 'Legalraasta', 'Lenden Club': 'Lendenclub',
    'Lending Kart': 'Lendingkart', 'Little Black Book Delhi': 'Little Black Book', 'Loan Tap': 'Loantap',
    'Log 9 Materials': 'Log9 Materials', 'Logicroots': 'Logic Roots', 'Milk Basket': 'Milkbasket', 
    'Nearbuy (Previously Groupon India)': 'Nearbuy', 'Neogrowth Credit': 'Neogrowth', 'Olacabs': 'Ola Cabs',
    'Oyorooms': 'Oyo Rooms', 'Oyo': 'Oyo Rooms', 'Paytm Marketplace': 'Paytm', 'Pinelabs': 'Pine Labs', 
    'Pipabella': 'Pipa Bella', 'Policy Bazaar': 'Policybazaar', 'Pressplay Tv': 'Pressplay', 'Qyk App': 'Qyk', 
    'Rapido Bike Taxi': 'Rapido', 'Rawpressery':'Raw Pressery', 'Renew Buy': 'Renewbuy', 'Rentmojo': 'Rentomojo',
    'Rollmafia': 'Roll Mafia', 'Satvakart': 'Satvacart', 'Shabdnagari': 'Shabdanagari', 'Shubhloans': 'Shubh Loans',
    'Silvan': 'Silvan Innovation Labs', 'Samunnati Financial Intermediation & Services Pvt. Ltd': 'Samunnati',
    'Spares Hub': 'Spareshub', 'Sport Flashes': 'Sports Flashes', 'Stalk Buy Love': 'Stalkbuylove',
    'Staydobe': 'Stayabode', 'Tails Life': 'Tailslife', 'Tempgo': 'Tempogo', 'The Mons Co.': 'The Moms Co',
    'Toko Innovations': 'Toko Innovation Studios', 'Tone Tag': 'Tonetag', 'Ue Life Sciences': 'Ue Lifesciences',
    'Unaacademy': 'Unacademy', 'Urbanclap Technologies Pvt. Ltd': 'Urbanclap', 'Vahdam Tea': 'Vahdam Teas',
    'Veritas Finance': 'Veritas Finance Ltd.', 'Vogo': 'Vogo Automotive', 'Vogo Automotive Pvt. Ltd.': 'Vogo Automotive',
    'Zippserve': 'Zippserv', 'Zolo': 'Zolostays'   
}
df['startup_name'] = df['startup_name'].replace(startup_mapping)

In [14]:
df.head()

,serial_no,disbursement_date,startup_name,industry_vertical,city_location,investors_name,investment_type,amount_in_usd
0,1,09/01/2020,Byjus,E-Tech,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000"
1,2,13/01/2020,Shuttl,Transportation,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394"
2,3,09/01/2020,Mamaearth,E-commerce,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860"
3,4,02/01/2020,Wealthbucket,FinTech,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000"
4,5,02/01/2020,Fashor,Fashion and Apparel,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000"


In [15]:
value_counts = df['startup_name'].value_counts().sort_index()
print(value_counts)

121Policy    1
19Th Mile    1
1Crowd       1
1Mg          5
20Dresses    1
            ..
Zupermeal    1
Zuppler      1
Zuver        1
Zwayam       1
Zzungry      1
Name: startup_name, Length: 2216, dtype: int64


### Cleaning the industry_vertical column

In [16]:
value_counts = df['industry_vertical'].value_counts().sort_index()
print(value_counts)

360-degree view creating platform                           1
3D Printer Manufacturer                                     1
AI                                                          1
API Workflow platform                                       1
Accounting                                                  1
                                                           ..
solar products and services marketplace                     1
us ticketing and fleet management platform                  1
usiness expense management                                  1
virtual reality, 3d simulation and stereoscopic products    1
visual search and discovery platform                        1
Name: industry_vertical, Length: 822, dtype: int64


In [17]:
#Function to replace partial matches based on the mapping
def replace_partial(text, mapping):
    for key, values in mapping.items():  # 'values' will be a list of keywords
        for value in values:  # Loop through each keyword in the list
            if re.search(value, text, re.IGNORECASE):  # Check for partial match (case-insensitive)
                return key  # Return the mapped category if a match is found
    return 'Others'  # Return the original if no match found

In [18]:
# Step 1: Clean the data (remove leading/trailing spaces, standardize case)
df['industry_vertical'] = df['industry_vertical'].str.strip().str.title()

category_mapping = {
    'Technology': ['AI', 'Machine Learning', 'Blockchain', 'Cloud', 'SaaS', 'Automation', 'Software', 'Platform', 'IT', 'Consumer Internet', 'Business Intelligence'],
    'E-Commerce & Retail': ['E-Commerce', 'Ecommerce', 'Retail', 'Online Store', 'Marketplace', 'Consumer Products', 'Fashion', 'Apparel', 'Consumer Goods'],
    'Health & Wellness': ['Health', 'Healthcare', 'Wellness', 'Medical', 'Fitness'],
    'Finance & Banking': ['Finance', 'Banking', 'Investment', 'Wealth', 'Insurance', 'Risk Management', 'Personal Finance', 'Fintech', 'Fin-Tech'],
    'Education & Learning': ['Education', 'E-Learning', 'Training', 'Learning', 'Ed-Tech', 'Edtech', 'Educational Technology'],
    'Transportation & Logistics': ['Transportation', 'Logistics', 'Mobility', 'Supply Chain', 'Vehicle', 'Transport'],
    'Entertainment & Media': ['Entertainment', 'Media', 'Streaming', 'Content', 'Games', 'Video'],
    'Food & Hospitality': ['Food', 'Restaurant', 'Hospitality', 'Travel', 'Hotel'],
    'Marketing & Advertising': ['Marketing', 'Advertising', 'Branding', 'Consumer Engagement'],
    'Energy & Environment': ['Energy', 'Sustainability', 'Clean Energy', 'Solar', 'Utilities'],
    'Real Estate': ['Real Estate', 'Property', 'Listings'],
    'Security & Privacy': ['Security', 'Privacy', 'Cybersecurity', 'Data Protection'],
    'Social & Community': ['Social', 'Network', 'Community', 'Engagement'],
    'Miscellaneous': ['Unknown']
}

# Apply the function to the 'industry_vertical' column
df['industry_vertical'] = df['industry_vertical'].apply(lambda x: replace_partial(x, category_mapping))

# Get value counts for the newly mapped 'industry_vertical' column
value_counts = df['industry_vertical'].value_counts()

# Display the count of each mapped category
print(value_counts)

Technology                    1342
Others                         703
E-Commerce & Retail            412
Miscellaneous                  171
Health & Wellness               98
Finance & Banking               81
Food & Hospitality              78
Transportation & Logistics      61
Education & Learning            57
Entertainment & Media           14
Real Estate                     12
Social & Community               7
Energy & Environment             6
Marketing & Advertising          2
Name: industry_vertical, dtype: int64


In [19]:
df.head()

,serial_no,disbursement_date,startup_name,industry_vertical,city_location,investors_name,investment_type,amount_in_usd
0,1,09/01/2020,Byjus,Others,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000"
1,2,13/01/2020,Shuttl,Transportation & Logistics,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394"
2,3,09/01/2020,Mamaearth,E-Commerce & Retail,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860"
3,4,02/01/2020,Wealthbucket,Finance & Banking,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000"
4,5,02/01/2020,Fashor,E-Commerce & Retail,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000"


In [20]:
value_counts = df['industry_vertical'].value_counts().sort_index()
print(value_counts)

E-Commerce & Retail            412
Education & Learning            57
Energy & Environment             6
Entertainment & Media           14
Finance & Banking               81
Food & Hospitality              78
Health & Wellness               98
Marketing & Advertising          2
Miscellaneous                  171
Others                         703
Real Estate                     12
Social & Community               7
Technology                    1342
Transportation & Logistics      61
Name: industry_vertical, dtype: int64


### Cleaning the city_location column

In [21]:
value_counts = df['city_location'].value_counts().sort_index()
print(value_counts)

Agra               2
Ahemadabad         2
Ahemdabad          1
Ahmedabad         38
Amritsar           1
                ... 
Udupi              2
Unknown          180
Uttar Pradesh      1
Vadodara          10
Varanasi           1
Name: city_location, Length: 108, dtype: int64


In [22]:
city_mapping = {
    "Ahemadabad": "Ahmedabad",
    "Ahemdabad": "Ahmedabad",
    "Andheri": "Mumbai",
    "Bangalore": "Bengaluru",
    "Bangalore / Palo Alto": "Bengaluru",
    "Bangalore / SFO": "Bengaluru",
    "Bangalore / San Mateo": "Bengaluru",
    "Bangalore / USA": "Bengaluru",
    "Bangalore/ Bangkok": "Bengaluru",
    "Belgaum": "Bengaluru",
    "Bengaluru and Gurugram": "Bengaluru",
    "Bhubneswar":"Bhubaneswar",
    "Chembur": "Mumbai",
    "Chennai/ Singapore": "Chennai",
    "Dallas / Hyderabad": "Hyderabad",
    "Delhi & Cambridge": "Delhi",
    "Goa/Hyderabad": "Goa",
    "Gurgaon": "Gurugram",
    "Gurgaon / SFO": "Gurugram",
    "Hyderabad/USA": "Hyderabad",
    "India / US": "India",
    "India/Singapore": "India",
    "India/US": "India",
    "Karnataka": "Bengaluru",
    "Kochi": "Kerala",
    "Kolkatta": "Kolkata",
    "Kormangala": "Bengaluru",
    "Kozhikode": "Kerala",
    "Menlo Park": "California",
    "Mumbai/Bengaluru": "Mumbai",
    "Mumbai / Global": "Mumbai",
    "Mumbai / NY": "Mumbai",
    "Mumbai / UK": "Mumbai",
    "New Delhi / California": "New Delhi",
    "New Delhi / US": "New Delhi",
    "New Delhi/ Houston": "New Delhi",
    "Nw Delhi": "New Delhi",
    "New York": "New York",
    "New York, Bengaluru": "New York",
    "New York/ India": "New York",
    "Noida / Singapore": "Noida",
    "Palo Alto": "California",
    "Panaji": "Goa",
    "Pune / Dubai": "Pune",
    "Pune / Singapore": "Pune",
    "Pune / US": "Pune",
    "Pune/Seattle": "Pune",
    "San Francisco": "California",
    "San Jose,": "California",
    "Santa Monica": "California",
    "SFO / Bangalore": "Bengaluru", 
    "Seattle / Bangalore": "Bengaluru",
    "Taramani": "Chennai",
    "Trivandrum": "Kerala",
    "US": "USA",
    "US/India": "USA",
    "USA/India": "USA"
}
df['city_location'] = df['city_location'].replace(city_mapping)

In [23]:
value_counts = df['city_location'].value_counts().sort_index()
print(value_counts)

Agra               2
Ahmedabad         41
Amritsar           1
Bengaluru        856
Bhopal             3
Bhubaneswar        2
Boston             1
Burnsville         1
California         7
Chandigarh        11
Chennai           99
Coimbatore         5
Delhi             35
Faridabad          3
Gaya               1
Goa               12
Gurugram         342
Gwalior            2
Haryana            1
Hubli              1
Hyderabad        101
India              5
Indore            13
Jaipur            30
Jodhpur            1
Kanpur             4
Karur              1
Kerala             6
Kolkata           22
London             1
Lucknow            1
Missourie          1
Mumbai           575
Nagpur             3
Nairobi            1
New Delhi        429
New York           3
Noida             94
Pune             112
Rourkela           1
Siliguri           2
Singapore          8
Srinagar           1
Surat              2
Tulangan           1
USA                4
Udaipur            2
Udupi        

### Cleaning the investors_name column

In [24]:
value_counts = df['investors_name'].value_counts().sort_index()
print(value_counts)

"Kedaraa Capital, Ontario Teachers\'"          1
021 Capita, Binny Bansal                       1
1Crowd                                         2
1Crowd (through crowd funding)                 1
1Crowd (through crowdfunding)                  1
                                              ..
ru-Net, Sequoia Capital, Lightbox Ventures     1
undisclosed investor                           3
undisclosed investors                         11
undisclosed private investors                  1
vCommission                                    3
Name: investors_name, Length: 2408, dtype: int64


In [25]:
def clean_investor_names(name):
    name = name.strip()  # Remove trailing spaces
    name = name.replace("\\", "")  # Remove backslashes
    name = name.split("(")[0].strip()  # Remove text in parentheses

    name = re.sub(r"\"?Kedaraa Capital, Ontario Teachers'\"?", "Kedara Capital, Ontario Teachers", name)
    name = re.sub(r"Undisclosed.*", "Unknown", name)
    return name

# Apply the combined cleaning function
df['investors_name'] = df['investors_name'].apply(clean_investor_names)

In [26]:
investors_mapping = {
    'Aarin Capital Partners,': 'Aarin Capital Partners',
    'Accel Partners,': 'Accel Partners',
    '3 undisclosed investors': 'Unknown', 
    '3 undisclosed HNIs': 'Unknown',
    'undisclosed private investors': 'Unknown',
    'undisclosed investors': 'Unknown',
    'undisclosed investor': 'Unknown',
    'Unnamed investor': 'Unknown',
    'Unnamed angel investors': 'Unknown',
    'Unnamed Media house and HNIs': 'Unknown',
    'Unknown High Net-Worth Individuals': 'Unknown',
    'Not Disclosed': 'Unknown',
    '3One4 Capital': '3one4 Capital',
    'Ajay Relan,': 'Ajay Relan',
    'Ant Financial': 'Ant Financial Services',
    'Ant Financials': 'Ant Financial Services',
    'BlackSoil Capital Pvt Ltd': 'BlackSoil Capital Pvt. Ltd.',
    'BlackSoil Capital Pvt. Ltd': 'BlackSoil Capital Pvt. Ltd.',
    'Blacksoil Capital Pvt. Ltd': 'BlackSoil Capital Pvt. Ltd.',
    'CDC Group plc': 'CDC Group Plc',
    'Carpediem Capital & Others': 'Carpediem Capital',
    'Catamaran Ventures, New India Investment Corporation': 'Catamaran Ventures',
    'Cataraman Ventures': 'Catamaran Ventures',
    'Chandigarh Angel Network':'Chandigarh Angels Network',
    'Chandigarh Angels Funding': 'Chandigarh Angels Network',
    'Cyber Carrier': 'Cyber Carrier CL',
    'DMG Information':'DMG information Asia Pacific',
    'DSG Consumer Partners, Saama Capital.': 'DSG Consumer Partners, Saama Capital',
    'Dheeraj Jain & Other angel investors': 'Dheeraj Jain & Others',
    'Dunamis Ventures Pte Ltd.': 'Dunamis Ventures Pte Ltd',
    'Eight Roads': 'Eight Roads Ventures',
    'Eight Road Ventures': 'Eight Roads Ventures',
    'Eight Roads Ventures India': 'Eight Roads Ventures',
    'Equanimity Venture Fund': 'Equanimity Ventures',
    'Facebook FB Start programme': 'Facebook FBStart',
    'Fairfax': 'Fairfax Holdings',
    'Gray Matters': 'Gray Matters Capital',
    'Green House Ventures Accelerator': 'Green House Ventures',
    'Group of Angel investors': 'Group of Investors',
    'Group of 15 Angel Investors': 'Group of Investors',
    'Group of Angel Investors': 'Group of Investors',
    'Group of Angel Investors from US/India': 'Group of Investors',
    'Group of HNI investors': 'Group of Investors',
    'Group of angel investors': 'Group of Investors',
    'Group of undisclosed Angel Investors & HNIs': 'Group of Investors',
    'Un disclosed investors': 'Unknown',
    'Helion Ventures Partners': 'Helion Venture Partners',
    'IFMR Capital Finance': 'IFMR Capital',
    'IdeaSpring Capital': 'Ideaspring Capital',
    'India Quotient and others': 'India Quotient & other investors',
    'Indian Angel network': 'Indian Angel Network',
    'Indian Angels Network': 'Indian Angel Network',
    'Individual investors': 'Individual Investors',
    'InfoEdge': 'Info Edge',
    'Infuse Ventures fund': 'Infuse Ventures',
    'Innoven Capital': 'InnoVen Capital',
    'International Financial Corporation': 'International Finance Corporation',
    'Kalpvriksh': 'Kalpavriksh',
    'LightBox Ventures': 'Lightbox Ventures',
    'Lightbox': 'Lightbox Ventures',
    'Lionrock Capital': 'LionRock Capital',
    'MakeMyTrip Limited': 'MakeMyTrip',
    'Michael and Susan Dell Foundation': 'Michael & Susan Dell Foundation',
    'Michael and Susan Dell Foundation, Anand Mahindra': 'Michael & Susan Dell Foundation, Anand Mahindra',
    'Micromax Informatics': 'Micromax Informatics Ltd',
    'Mount Nathan Advisors Pte Ltd': 'Mount Nathan Advisors',
    'Multiple Angel Investors': 'Group of Investors',
    'Multiple investors': 'Group of Investors',
    'Mumbai Angels & Other angel investors': 'Mumbai Angels & Group of other investors',
    'Nihon Kotsu Co': 'Nihon Kotsu Co. Ltd', 
    'Nihon Kotsu': 'Nihon Kotsu Co. Ltd',
    'Oliphabs Capital': 'Oliphans Capital',
    'Omidyar Network,': 'Omidyar Network',
    'One 97 Communications': 'One97 Communications Ltd.',
    'One97 Communication Ltd': 'One97 Communications Ltd.',
    'One97 Communications': 'One97 Communications Ltd.',
    'Prime Venture Partners,': 'Prime Venture Partners',
    'Quona Capital Management': 'Quona Capital',
    'Rata Tata': 'Ratan Tata',
    'Reliance Capital Venture': 'Reliance Capital',
    'RoundGlass Partners LLC': 'RoundGlass Partners',
    'S Chand & Co': 'S Chand and Co Pvt Ltd',
    'SAIF Partners & others': 'SAIF Partners and others',
    'SAIF partners': 'SAIF Partners',
    'Saif Partners': 'SAIF Partners',
    'SIDBI': 'SIDBI Venture Capital Ltd',
    'SIDBI Venture Capital Fund': 'SIDBI Venture Capital Ltd',
    'Scale Ventures Funds': 'Scale Ventures',
    'SoftBank Group Corp': 'SoftBank Group',
    'SoftBank Corp': 'SoftBank Group',
    'Softbank Group': 'SoftBank Group',
    'Sprout Angels': 'Sprout Angels LLP',
    'Star India Pvt. Ltd': 'Star India Pvt Ltd',
    'Stellaris Venture Partners,': 'Stellaris Venture Partners',
    'Susquehanna Growth Equity LLC': 'Susquehanna Growth Equity',
    'Swastika Company Ltd.': 'Swastika Company Ltd',
    'Swastika': 'Swastika Company Ltd',
    'Swastika Co Ltd.': 'Swastika Company Ltd',
    'Swastika Company Pvt Ltd': 'Swastika Company Ltd',
    'Swastika Investmart Ltd': 'Swastika Company Ltd',
    'Trifecta Capital Advisors': 'Trifecta Capital',
    'Tekne Capital Management': 'Tekne Capital Management LLC',
    'Times Group': 'Times Group Brand Capital',
    'Times Internet Ltd': 'Times Internet',
    'Unicorn Venture': 'Unicorn India Ventures',
    'Unicorn Ventures': 'Unicorn India Ventures',
    'Unilazer': 'Unilazer Ventures',
    'Unilazer Ventures,': 'Unilazer Ventures',
    'Uniliver Ventures': 'Unilever Ventures',
    'Viraj Tyagi & others': 'Viraj Tyagi & Others',
    'WaterBridge Ventures,': 'WaterBridge Ventures',
    'Westbridge Capital': 'WestBridge Capital',
    'Westbridge Capital Partners': 'WestBridge Capital',
    'YouWeCan Ventures,': 'YouWeCan Ventures',
    'YourNest Angel Fund,': 'YourNest Angel Fund'
}
df['investors_name'] = df['investors_name'].replace(investors_mapping)

### Cleaning the investment_type column

In [27]:
value_counts = df['investment_type'].value_counts().sort_index()
print(value_counts)

Angel                             1
Angel / Seed Funding              8
Angel Funding                     1
Angel Round                       1
Bridge Round                      1
Corporate Round                   2
Crowd Funding                     1
Crowd funding                     1
Debt                              1
Debt Funding                     25
Debt and Preference capital       1
Debt-Funding                      1
Equity                            2
Equity Based Funding              1
Funding Round                     1
Inhouse Funding                   1
Maiden Round                      1
Mezzanine                         1
Pre Series A                      1
Pre-Series A                      4
Pre-series A                      1
Private                           1
Private Equity                 1356
Private Equity Round              4
Private Funding                   1
PrivateEquity                     2
Seed                              4
Seed / Angel Funding        

In [28]:
funding_mapping = {
    'Angel': 'Angel Funding',
    'Angel / Seed Funding': 'Angel Seed Funding',
    'Angel Round': 'Angel Funding',
    'Bridge Round': 'Bridge Funding',
    'Corporate Round': 'Corporate Funding',
    'Crowd Funding': 'Crowdfunding',
    'Crowd funding': 'Crowdfunding',
    'Debt': 'Debt Funding',
    'Debt-Funding': 'Debt Funding',
    'Debt and Preference capital': 'Debt Funding',
    'Equity': 'Equity Funding',
    'Equity Based Funding': 'Equity Funding',
    'Funding Round': 'Other Funding',
    'Inhouse Funding': 'Other Funding',
    'Maiden Round': 'Other Funding',
    'Mezzanine': 'Other Funding',
    'Pre Series A': 'Pre-Series A',
    'pre-series A': 'Pre-Series A',
    'pre-Series A': 'Pre-Series A',
    'Pre-series A': 'Pre-Series A',
    'Private': 'Private Equity',
    'Private Equity Round': 'Private Equity',
    'Private Funding': 'Private Equity',
    'PrivateEquity': 'Private Equity',
    'Seed': 'Seed Funding',
    'Seed / Angel Funding': 'Angel Seed Funding',
    'Seed / Angle Funding': 'Angel Seed Funding',
    'Seed Funding Round': 'Seed Funding',
    'Seed Round': 'Seed Funding',
    'Seed funding': 'Seed Funding',
    'Seed/ Angel Funding': 'Angel Seed Funding',
    'Seed/Angel Funding': 'Angel Seed Funding',
    'SeedFunding': 'Seed Funding',
    'Series B (Extension)':'Series B',
    'Single Venture': 'Venture Funding',
    'Structured Debt': 'Debt Funding',
    'Term Loan': 'Debt Funding',
    'Venture': 'Venture Funding',
    'Venture - Series Unknown': 'Venture Funding',
    'Venture Round': 'Venture Funding'
}

df['investment_type'] = df['investment_type'].replace(funding_mapping)

In [29]:
value_counts = df['investment_type'].value_counts().sort_index()
print(value_counts)

Angel Funding            3
Angel Seed Funding     141
Bridge Funding           1
Corporate Funding        2
Crowdfunding             2
Debt Funding            30
Equity Funding           3
Other Funding            4
Pre-Series A             9
Private Equity        1364
Seed Funding          1398
Series A                24
Series B                21
Series C                14
Series D                12
Series E                 2
Series F                 2
Series G                 1
Series H                 1
Series J                 1
Unknown                  4
Venture Funding          5
Name: investment_type, dtype: int64


### Cleaning the amount column

In [30]:
def clean_amount(amount):
    amount = str(amount).strip().lower() #Converting to string and stripping spaces
    if amount in ['n/a', 'undisclosed', 'unknown']: 
        return 0.0

    #Regex to remove unwanted characters (except for digits, decimal point, and minus sign)
    amount = re.sub(r'[^0-9.-]', '', amount)
    try:
        cleaned_amount = float(amount)
    except ValueError:
        cleaned_amount = 0.0
    return cleaned_amount

df['amount_in_usd'] = df['amount_in_usd'].apply(clean_amount)

#Formatting the output to avoid scientific notation and keeping it as float
pd.set_option('display.float_format', '{:,.2f}'.format)
df.head()

,serial_no,disbursement_date,startup_name,industry_vertical,city_location,investors_name,investment_type,amount_in_usd
0,1,09/01/2020,Byjus,Others,Bengaluru,Tiger Global Management,Private Equity,"200,000,000.00"
1,2,13/01/2020,Shuttl,Transportation & Logistics,Gurugram,Susquehanna Growth Equity,Series C,"8,048,394.00"
2,3,09/01/2020,Mamaearth,E-Commerce & Retail,Bengaluru,Sequoia Capital India,Series B,"18,358,860.00"
3,4,02/01/2020,Wealthbucket,Finance & Banking,New Delhi,Vinod Khatumal,Pre-Series A,"3,000,000.00"
4,5,02/01/2020,Fashor,E-Commerce & Retail,Mumbai,Sprout Venture Partners,Seed Funding,"1,800,000.00"


In [31]:
value_counts = df['amount_in_usd'].value_counts().sort_index()
print(value_counts)

0.00                971
16,000.00             5
16,600.00             6
18,000.00             1
20,000.00             1
                   ... 
700,000,000.00        1
1,000,000,000.00      1
1,400,000,000.00      2
2,500,000,000.00      1
3,900,000,000.00      1
Name: amount_in_usd, Length: 458, dtype: int64


In [32]:
df.sample(100)

,serial_no,disbursement_date,startup_name,industry_vertical,city_location,investors_name,investment_type,amount_in_usd
1120,1121,02/12/2016,Tygr,Others,Mumbai,Franchise India,Private Equity,"3,000,000.00"
932,933,02/03/2017,Creditmantri,Technology,Chennai,Quona Capital,Private Equity,"7,600,000.00"
300,301,19/04/2018,Mamaearth,Health & Wellness,Gurugram,Shilpa Shetty Kundra,Private Equity,0.00
1402,1403,28/09/2016,Beardo,E-Commerce & Retail,Ahmedabad,Venture Catalysts,Seed Funding,"500,000.00"
1712,1713,18/05/2016,Fundamentor,Education & Learning,Pune,Subramanya SV,Seed Funding,0.00
...,...,...,...,...,...,...,...,...
586,587,07/09/2017,Oyo Rooms,Technology,Gurugram,"SoftBank Group, Sequoia Capital India, Lightsp...",Private Equity,"250,000,000.00"
1364,1365,12/09/2016,Nykaa,E-Commerce & Retail,Mumbai,"Sunil Munjal, Mariwala family",Private Equity,"12,300,000.00"
1804,1805,25/04/2016,Commut,Technology,Hyderabad,50K Ventures,Seed Funding,"200,000.00"
2458,2459,28/09/2015,Ola Cabs,Technology,Bengaluru,Didi Kuaidi,Private Equity,"30,000,000.00"
